# Drzewa decyzyjne #

Podczas wykonywania ćwiczenia będzie wykorzystywany pakiet scikit-learn (sklearn), którego metody umożliwiają m.in. budowę drzew decyzyjnych na podstawie danych, testowanie drzew oraz przeprowadzanie klasyfikacji.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import matplotlib.pyplot as plt
iris = load_iris()  #wczytujemy przykladowy zbior danych

Tworzymy obiekt dla drzewa.

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
mytree1 = tree.DecisionTreeClassifier()

Uczymy klasyfikator na podstawie danych przykładowego zbioru danych iris (https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)

In [ ]:
mytree1.fit(iris.data, iris.target)

Korzystając z metody `get_params` można odczytać parametry konfiguracyjne drzewa.

In [ ]:
mytree1.get_params()

Klasyfikujemy przykładowy wektor cech.

In [ ]:
klasa = mytree1.predict([[2,3,4,5]])
print(iris.target_names[klasa])

Można też wyznaczyć prawdopodobieństwa przynależności do poszczególnych klas.

In [ ]:
mytree1.predict_proba([[2,3,4,5]])

# Struktura drzewa

Funkcja `plot_tree` umożliwia narysowanie grafu prezentującego skonstruowane drzewo decyzyjne.

In [ ]:
plt.figure(figsize=(12,12))
tree.plot_tree(mytree1, filled=True, feature_names=iris.feature_names, class_names=iris.target_names, fontsize = 8)
plt.show()

Struktura drzewa zapisana jest w atrybucie `tree_`. Jest tam informacja o liczbie węzłów oraz tablice, w których dla każdego węzła zapisane są

* w przypadku węzłów wewnętrznych: indeks lewego i prawego potomka, indeks wybranego atrybutu i wartość progowa

* w przypadku liści: inne wartości, w tym wypadku nia mające znaczenia

Przeanalizuj sposób zapisu struktury drzewa mytree1.

In [ ]:
print("Liczba wezlow: ", mytree1.tree_.node_count)
print("Indeksy lewych potomkow:", mytree1.tree_.children_left)
print("Indeksy prawych potomkow:",mytree1.tree_.children_right)
print("Wybrane atrybuty:", mytree1.tree_.feature)
print("Wartosci progowe:", mytree1.tree_.threshold)

# Testowanie drzewa
**Na podstawie odrębnego zbioru danych** 

Zbiór danych jest dzielony na dane uczące i testowe (`train_test_split`). Metoda `score` zwraca poprawność klasyfikacji. Korzystając z funkcji `confusion_matrix` można dowiedzieć się dokładnie jakie są wyniki klasyfikacji w zależności od klasy. Funkcja ta zwraca tzw. macierz błędów, której element (i,j) zawiera liczbę przykładów klasy i-tej zaklasyfikowanych do klasy j-tej.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
data_train, data_test, target_train, target_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)
mytree1.fit(data_train, target_train)
print("Zaklaasyfikowano poprawnie", mytree1.score(data_test,target_test), "przykladow ze ze zbioru testowego.")
confusion_matrix(target_test, mytree1.predict(data_test))

**W procesie walidacji krzyżowej** 

Zbiór danych jest dzielony na k podzbiorów. Uczenie i testowanie odbywa się k razy. W każdej iteracji 1 z k podzbiorów zostaje wykorzystany do testowania a wszystkie pozostałe do uczenia. Wyniki zostają uśrednione (`cross_val_score`). 

In [ ]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(mytree1, iris.data, iris.target, cv=5)

Otrzymujemy wyniki testowania z poszczególnych przebiegów walidacji a także wynik uśredniony wraz z 95%-owym przedziałem ufności. Rzeczywisty wynik mieści się w tym przedziale z prawdopodobieństwem 0,95.

In [ ]:
print("Poprawosc klasyfikacji w kolejnych iteracjach: ", results)
print("Poprawnosc klasyfikacji: %0.2f (+/- %0.2f)" % (results.mean(), results.std() * 2))

W wyniku walidacji krzyżowej każdy przykład jeden raz trafia do zbioru testowego. Wyniki predykcji dla poszczególnych przykładów można odczytać korzystając z funkcji `cross_val_predict`. Funkcja ta, podobnie jak `cross_val_score` przeprowadza waidację krzyżową, ale zwraca wyniki klasyfikacji poszczególnych przykładów zamiast poprawności w kolejnych iteracjach.

In [ ]:
from sklearn.model_selection import cross_val_predict
cross_val_predict(mytree1, iris.data, iris.target, cv=5)

**Zadanie 1 (1 pkt.):**

Podaj liczbę oraz indeksy przykładów błędnie zaklasyfikowanych podczas 10-krotnej walidacji krzyżowej.

**Zadanie 2 (0,5 pkt.):**

Pzetestuj przeprowadzając 10-krotną walidację krzyżową drzewa różniące się zastosowanym kryterium wyboru atrubutu: przyrost informacji (`criterion = 'entropy'`) lub wskaźnik Gini (`criterion = 'gini'`). Porównaj jakość klasyfikacji w obu przypadkach.

# Ograniczanie rozmiarów drzewa - kryterium stopu

Możemy ograniczyć rozmiar drzewa ustawiając wartość jednego z następujących parametrów:

`min_samples_split` - minimalna liczba przykładów wymagana, aby dane w węźle dalej dzielić

`min_samples_leaf` - minimalna dopuszczalna liczba przykładów w liściu

`max_leaf_nodes` - maksymalna liczba liści

`max_depth` - maksymalna głębokość drzewa

Poeksperymentuj z drzewem uczonym na podstawie zbioru iris ograniczając jego rozmiary za pomocą powyższych parametrów. Efekt działania obserwuj rysując drzewo.

**Zadanie 3 (0,5 pkt.):**

Zapoznaj się ze zbiorem danych digits:

http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

Do wczytania danych wykorzystaj funkcję load_digits().

Zmodyfikuj drzewo mytree1 tak, aby minimalna liczba przykładów w liściu wynosiła 4. Zmodyfikuj drzewo mytree2 tak, aby nie miało ono więcej niż 5 liści. Porównaj teraz dokładność klasyfikacji obu drzew dla danych ze zbioru uczącego.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()


**Zadanie 4 (2 pkt):**

Dobierz optymalną wartość współczynnika `max_leaf_nodes` dla zbioru `digits`. W tym celu podziel zbiór danych `digits` na dwie części. Korzystając z pierwszego podzbioru przeprowadź walidację krzyżową wielokrotnie dla różnych wartości optymalizowanego współczynnika. Zidentyfikuj wartość optymalną, czyli taką, dla której średni błąd jest najmniejszy (szacowany w procesie walidacji krzyżowej). Po znalezieniu optymalnej wartości parametru naucz drzewo dla znalezionej wartości `max_leaf_nodes`. Tym razem do uczenia wykorzystaj cały pierwszy podzbiór danych. Następnie przetestuj skonstruowane drzewo na danych z drugego podzbioru. Porównaj otrzymane wyniki z wynikami otrzymanymi przed optymalizacją parametru `max_leaf_nodes`. Rozwiązując zadanie wyświetlaj wszystkie pośrednie wyniki (nie tylko końcowe).


**Zadanie 5 (2 pkt):**

Podziel zbiór danych na uczące i testowe w stosunku 2:1. Przeprowadź uczenie i testowanie wielokrotnie zwiększając w kolejnych krokach wartość parametru `max_leaf_nodes`. Narysuj wykres pokazujący zależność błędu klasyfikacji od rozmiarów drzewa. Zależność tę pokaż zarówno dla danych testowych, jak i uczących.

# Lasy losowe #

Zapoznaj się z opisem klasy `RandomForestClassifier`

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**Zadanie 6 (1,5 pkt.)**

Dla zbioru danych digits skonstruuj las losowy zawierający 20 drzew. 

a) Jaki jest błąd klasyfikacji dla zbioru uczącego?

b) Jaki jest błąd klasyfikacji oszacowany w procesie walidacji krzyżowej?

c) Która cyfra (klasa) jest najlepiej rozpoznawana a która najsłabiej?

d) Jakiego rodzaju błąd jest najczęściej popełniany (która cyfra z którą jest najczęściej mylona)?

**Zadanie 7 (1 pkt.)**
Dla każdej klasy podaj poprawność klasyfikacji, czyli stosunek poprawnie zaklasyfikowanych przykładów danej klasy do wszystkich przykładów danej klasy. Jest to jeden z parametrów jakości klasyfikatora, tzw. recall. Można w tym celu wykorzystać funkcję `recall_score` lub wyznaczyć odpowiednie wartości samodzielnie korzystając z macierzy błędów (confusion matrix). Wyznacz wartości recall na dwa sposoby.

**Zadanie 8 (1,5 pkt.)**

Wykreśl zależność błędu klasyfikacji lasu losowego od liczby drzew w lesie. Błąd klasyfikacji szacuj w procesie 5-krotnej walidacji krzyżowej.